In [1]:
import numpy as np
import scipy.misc
from keras import backend as K
# from keras import initializations
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

Using TensorFlow backend.


In [38]:
train_data = np.load('train_cats_and_dogs_images.npz')['arr_0'].reshape([-1, 150, 150, 3])
# the features were saved in order, so recreating the labels is easy
train_labels = np.array([0] * 1000 + [1] * 1000)

test_data = np.load('test_cats_and_dogs_images.npz')['arr_0'].reshape([-1, 150, 150, 3])
test_labels = np.array([0] * 400 + [1] * 400)

In [39]:
train = np.zeros([2000, 150, 150, 3])
for i in range(2000):
    train[i] = train_data[i]

In [40]:
test = np.zeros([800, 150, 150, 3])
for i in range(800):
    test[i] = test_data[i]

In [41]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

In [42]:
train_datagen.fit(train)
test_datagen.fit(test)

In [55]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [56]:
model.fit_generator(train_datagen.flow(train, train_labels, batch_size=16),
                    samples_per_epoch=train.shape[0],
                    nb_epoch=30, 
                    validation_data=test_datagen.flow(test, test_labels))

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  after removing the cwd from sys.path.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_data=<keras.pre..., steps_per_epoch=125, epochs=30)`
  after removing the cwd from sys.path.


Epoch 1/30
125/125 [==============================] - 112s 894ms/step - loss: 0.7091 - acc: 0.5370 - val_loss: 0.6868 - val_acc: 0.5437
Epoch 2/30
125/125 [==============================] - 109s 869ms/step - loss: 0.6909 - acc: 0.5515 - val_loss: 0.6529 - val_acc: 0.6238
Epoch 3/30
125/125 [==============================] - 112s 899ms/step - loss: 0.6565 - acc: 0.6250 - val_loss: 0.5992 - val_acc: 0.6875
Epoch 4/30
125/125 [==============================] - 106s 850ms/step - loss: 0.6277 - acc: 0.6630 - val_loss: 0.5740 - val_acc: 0.6937
Epoch 5/30
125/125 [==============================] - 109s 869ms/step - loss: 0.6056 - acc: 0.6770 - val_loss: 0.5658 - val_acc: 0.7063
Epoch 6/30
125/125 [==============================] - 108s 861ms/step - loss: 0.5995 - acc: 0.6930 - val_loss: 0.6073 - val_acc: 0.6813
Epoch 7/30
125/125 [==============================] - 105s 837ms/step - loss: 0.5629 - acc: 0.7225 - val_loss: 0.5605 - val_acc: 0.7000
Epoch 8/30
125/125 [============================

In [57]:
model.save_weights('first_try.h5')  # always save your weights after training or during training